# Batch demand calculation

Load the dwelling summary, slice the EOH2303 weather profile for 9–13 February 2022, dispatch the hybrid heat pump model for a single dwelling, and visualise the result with the reusable plotting helper from `RC_Optimization.py`.


In [ ]:
from pathlib import Path
import sys
import pandas as pd

# Ensure local source files are importable when the notebook is run interactively
SRC_DIR = Path.cwd()
if str(SRC_DIR) not in sys.path:
    sys.path.append(str(SRC_DIR))
PARENT_DIR = SRC_DIR.parent
if str(PARENT_DIR) not in sys.path:
    sys.path.append(str(PARENT_DIR))

from RC_Optimization import build_tariff, optimize_hhp_operation, plot_hhp_results_components
from generate_demand_profiles import (
    load_summary,
    load_weather,
    extract_outdoor_temperature,
    extract_solar_gain,
    derive_setpoint,
    derive_initial_temperature,
    ensure_aligned_tariff,
)


In [ ]:
# File locations and analysis window
summary_path = Path("../Data/1R1C1P1S_filtered.csv")
weather_path = Path("../Data/EOH2303_merged_30min.csv")
start = pd.Timestamp("2022-02-09 00:00")
end = pd.Timestamp("2022-02-13 23:30")

params = load_summary(summary_path)
weather = load_weather(weather_path, start, end)

dt_seconds = weather.index.to_series().diff().dropna().median().total_seconds()

# Build and align tariff data with the weather timestamps
n_days = int((end.normalize() - start.normalize()).days) + 1
tariff = build_tariff(start.normalize(), n_days=n_days, step="30min")
tariff = ensure_aligned_tariff(tariff, weather.index)

# Prepare optimisation inputs
Tout = extract_outdoor_temperature(weather)
S = extract_solar_gain(weather, dt_seconds)
setpoint = derive_setpoint(weather.index)
T0 = derive_initial_temperature(weather)


In [ ]:
# Run the optimisation for the first dwelling in the summary
first_dwelling = params[0]
results = optimize_hhp_operation(
    first_dwelling.R1,
    first_dwelling.C1,
    first_dwelling.g,
    tariff,
    Tout,
    S,
    dt_seconds,
    T0,
    T_setpoint=setpoint,
    tol=1.0,
    COP=3.5,
    etaB=0.9,
    Qhp_max=4_000.0,
    Qbo_max=24_000.0,
)

results.head()


In [ ]:
# Visualise the dispatch using the shared plotting helper
plot_hhp_results_components(
    Tin=results["Tin"].to_numpy(),
    Q_hp=results["Q_hp"].to_numpy(),
    Q_bo=results["Q_bo"].to_numpy(),
    T_set=results["T_set"].to_numpy(),
    T_low=results["T_low"].to_numpy(),
    T_high=results["T_high"].to_numpy(),
    index=results.index,
    tariff=tariff,
    df=weather,
    title_prefix=f"{first_dwelling.dwelling_id} dispatch (9–13 Feb 2022)",
    filename=f"{first_dwelling.dwelling_id}_dispatch.png",
)
